In [2]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

In [1]:
from re import T
import matplotlib
import matplotlib.pyplot as plt
from tools.utils import EarlyStopping
from tools.training_tools import test
from tools.metrics import RMSE
from tools.evaluate_tools import evaluate, evaluate_adrnn_cnn, \
    evaluate_adrnn_cnn_multilabel, evaluate_separate_lstm, evaluate_simulate,evaluate_simu_t, evaluate_stacked_lstm, evaluate_flat_lstm,evaluate_timesetps
from tools.draw_lines import draw_lines
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn as nn
import torch
import numpy as np
import random
import string
from ast import Try
from cProfile import label
import codecs
import imp
import os
from turtle import color

from sklearn import metrics

from nets.transformer import StackedTransformer,Transformer
from tools.training_tools import test_informer, train_informer, train_stacked_lstm_cnn,test
from tools.simulation import simulation, simulation_transformer
from data.data_process import dataset_split, dataset_split_h
from data.sepsis_dataset import (SepsisDataset, TestDataset, TrainDataset,
                                 VaildDataset)
from torch.utils.data import DataLoader

In [3]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 0 
          }

dataset, mask = torch.load('./datasets/normalized_dataset_new.pt')
train_data, train_mask, valid_data, valid_mask, test_data, test_mask = dataset_split_h(
    dataset, mask)  # b t input

train_dataset = TrainDataset(train_data, train_mask)
vaild_dataset = VaildDataset(valid_data, valid_mask)
test_dataset = TestDataset(test_data, test_mask)

train_iterator= DataLoader(train_dataset,**params)
vaild_iterator=DataLoader(vaild_dataset,**params)
test_iterator=DataLoader(test_dataset,**params)      

In [24]:
class GRU(nn.Module):
    def __init__(self,hidden_size=64, num_layers=2, dropout=0.2,input_size=49):
        super(GRU, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.input_size = input_size
        self.rnn = nn.GRU(
            input_size=input_size,
            hidden_size=self.hidden_size,     # rnn hidden unit
            num_layers=self.num_layers,       # number of rnn layer
            dropout=self.dropout,
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.drop_layer = nn.Dropout(p=self.dropout)
        self.out = nn.Linear(self.hidden_size, input_size)

    def forward(self, x, h_state=None, is_train=True, gen_length=None,bias=None):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, hidden_size)
        start=12
        batch = x.shape[0]
        r_out, h_state = self.rnn(x, h_state)
        #return r_out, h_state
        r_out = self.drop_layer(r_out)
        r_out = self.out(r_out) #-> b t inputsize
        if is_train==False:
            r_output = torch.zeros(gen_length, batch, self.input_size).cuda()  #   t b input_size
            r_out = x.permute(1,0,2)[start-1] #->  b hidden_size
            r_output[:start-1,:,:] = x.permute(1,0,2)[:start-1,:,:]
            r_out = r_out.view(batch, 1, -1) #->  b 1 hidden_size
            for i in range(start-1,gen_length-1):
                r_out, h_state = self.rnn(r_out, h_state) #r_out  b 1 hidden_size
                r_out = self.drop_layer(r_out)
                r_out = self.out(r_out) #-> b t inputsize
                if bias !=None:
                    r_out[:,:,:47] = r_out[:,:,:47] +bias[:,i-(start-1):i-(start-2),:]
                r_output[i+1]= r_out.permute(1,0,2)[0]

            return r_output.permute(1,0,2), h_state
        return r_out, h_state

In [25]:
class RNN(nn.Module):
    def __init__(self,hidden_size=64, num_layers=2, dropout=0.2,input_size=49):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.input_size = input_size
        self.rnn = nn.RNN(
            input_size=input_size,
            hidden_size=self.hidden_size,     # rnn hidden unit
            num_layers=self.num_layers,       # number of rnn layer
            dropout=self.dropout,
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.drop_layer = nn.Dropout(p=self.dropout)
        self.out = nn.Linear(self.hidden_size, input_size)

    def forward(self, x, h_state=None, is_train=True, gen_length=None,bias=None):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, hidden_size)
        batch = x.shape[0]
        start=12

        if is_train==False:
            r_output = torch.zeros(gen_length, batch, self.input_size).cuda()  #   t b input_size
            r_out = x.permute(1,0,2)[start-1] #->  b hidden_size
            r_output[:start-1,:,:] = x.permute(1,0,2)[:start-1,:,:]
            r_out = r_out.view(batch, 1, -1) #->  b 1 hidden_size
            for i in range(start-1,gen_length-1):
                r_out, h_state = self.rnn(r_out, h_state) #r_out  b 1 hidden_size
                r_out = self.drop_layer(r_out)
                r_out = self.out(r_out) #-> b t inputsize
                if bias !=None:
                    r_out[:,:,:47] = r_out[:,:,:47] +bias[:,i-(start-1):i-(start-2),:]
                r_output[i+1]= r_out.permute(1,0,2)[0]
            return r_output.permute(1,0,2), h_state
        r_out, h_state = self.rnn(x, h_state)
        #return r_out, h_state
        r_out = self.drop_layer(r_out)
        r_out = self.out(r_out) #-> b t inputsize        
        return r_out, h_state

In [26]:
class LSTM(nn.Module):
    def __init__(self,hidden_size=64, num_layers=2, dropout=0.2,input_size=49):
        super(LSTM, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.input_size = input_size
        self.rnn = nn.LSTM(
            input_size=input_size,
            hidden_size=self.hidden_size,     # rnn hidden unit
            num_layers=self.num_layers,       # number of rnn layer
            dropout=self.dropout,
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.drop_layer = nn.Dropout(p=self.dropout)
        self.out = nn.Linear(self.hidden_size, input_size)

    def forward(self, x, h_state=None, is_train=True, gen_length=None,bias=None):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, hidden_size)
        batch = x.shape[0]
        start=12
        if is_train==False:
            r_output = torch.zeros(gen_length, batch, self.input_size).cuda()  #   t b input_size
            r_out = x.permute(1,0,2)[start-1] #->  b hidden_size
            r_output[:start-1,:,:] = x.permute(1,0,2)[:start-1,:,:]
            r_out = r_out.view(batch, 1, -1) #->  b 1 hidden_size
            for i in range(start-1,gen_length-1):
                r_out, h_state = self.rnn(r_out, h_state) #r_out  b 1 hidden_size
                r_out = self.drop_layer(r_out)
                r_out = self.out(r_out) #-> b t inputsize
                if bias !=None:
                    r_out[:,:,:47] = r_out[:,:,:47] +bias[:,i-(start-1):i-(start-2),:]
                r_output[i+1]= r_out.permute(1,0,2)[0]

            return r_output.permute(1,0,2), h_state
        r_out, h_state = self.rnn(x, h_state)
        #return r_out, h_state
        r_out = self.drop_layer(r_out)
        r_out = self.out(r_out) #-> b t inputsize        
        
        return r_out, h_state

In [47]:
class LSTM2(nn.Module):
    def __init__(self,hidden_size=64, num_layers=2, dropout=0.2,input_size=47):
        super(LSTM2, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.input_size = input_size
        self.rnn1 = nn.LSTM(
            input_size=input_size,
            hidden_size=self.hidden_size,     # rnn hidden unit
            num_layers=self.num_layers,       # number of rnn layer
            dropout=self.dropout,
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.rnn2 = nn.LSTM(
            input_size=47,
            hidden_size=self.hidden_size,     # rnn hidden unit
            num_layers=self.num_layers,       # number of rnn layer
            dropout=self.dropout,
            batch_first=True,   # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.drop_layer = nn.Dropout(p=self.dropout)
        self.drop_layer2 = nn.Dropout(p=self.dropout)
        self.out = nn.Linear(self.hidden_size, 47)
        self.out2 = nn.Linear(self.hidden_size, 2)

    def forward(self, x, h_state=None,t_state=None, is_train=True, gen_length=None,bias=None):
        # x (batch, time_step, input_size)
        # h_state (n_layers, batch, hidden_size)
        # r_out (batch, time_step, hidden_size)
        batch = x.shape[0]
        gen_length=24
        x=x[:,0:gen_length,:]
        if is_train==False:
            for i in range(gen_length):
                Input=x[:,i:i+gen_length,:]
                r_out, h_state = self.rnn1(Input, h_state)
                #return r_out, h_state
                r_out = self.drop_layer(r_out)
                r_out = self.out(r_out) #-> b t inputsize
                t_out, t_state = self.rnn2(r_out[:,-1:,:], t_state)
                t_out = self.drop_layer2(t_out)
                t_out = self.out2(t_out)
                
                B_out=torch.cat((r_out[:,-1:,:],t_out[:,-1:,:]),dim=2)
                if bias!=None:
                    B_out[:,:,:47]=B_out[:,:,:47]+bias[:,i:i+1,:]
                x=torch.cat((x,B_out[:,-1:,:]),dim=1)
                
                    

            return x[:,:,:], h_state
        # batch = x.shape[0]
        
        
        for i in range(gen_length):
            Input=x[:,i:i+gen_length,:]
            r_out, h_state = self.rnn1(Input, h_state)
            #return r_out, h_state
            r_out = self.drop_layer(r_out)
            r_out = self.out(r_out) #-> b t inputsize
            t_out, t_state = self.rnn2(r_out[:,-1:,:], t_state)
            t_out = self.drop_layer2(t_out)
            t_out = self.out2(t_out)
            
            B_out=torch.cat((r_out[:,-1:,:],t_out[:,-1:,:]),dim=2)
            x=torch.cat((x,B_out[:,-1:,:]),dim=1)

        return x[:,:,:], h_state

In [48]:
def evaluate_simulation_rnn(model, data_x, data_y, mask, response_size=47, treatment_size=2,bias=None):
    
    
    length = data_x.shape[1]
    middle_len=24
    all_len=48
    loss_mse = nn.MSELoss()
    loss_mae = nn.L1Loss()

    pred,_ = model(data_x, is_train=False, gen_length=all_len,bias=bias) #1,74,49

    pred = torch.mul(pred[:,middle_len:all_len,:], mask[:,middle_len:all_len,:])

    eval_mse_r = loss_mse(data_y[:, middle_len:all_len, :response_size], pred[:, :, :response_size]).item()
    eval_mse_t = loss_mse(data_y[:, middle_len:all_len, -treatment_size:], pred[:, :, -treatment_size:]).item()
    eval_mae_r = loss_mae(data_y[:, middle_len:all_len, :response_size], pred[:, :, :response_size]).item()
    eval_mae_t = loss_mae(data_y[:, middle_len:all_len, -treatment_size:], pred[:, :, -treatment_size:]).item()
    bias=(data_y[:, middle_len:all_len, :response_size]-pred[:,:, :response_size])
    return eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t,bias,pred

In [29]:
def evaluate_simulation_rt(model_r,model_t, data_x, data_y, mask, response_size=47, treatment_size=2,bias=None):
    
    
    length = data_x.shape[1]
    all_len=48
    loss_mse = nn.MSELoss()
    loss_mae = nn.L1Loss()

    pred_r,_ = model_r(data_x[:,:,:47], is_train=False, gen_length=all_len,bias=bias) #1,74,49
    pred_t,_ = model_t(data_x[:,:,47:], is_train=False, gen_length=all_len) #1,74,49   

    pred_r = torch.mul(pred_r[:,24:48,:], mask[:,24:48,:47])
    pred_t = torch.mul(pred_t[:,24:48,:], mask[:,24:48,47:])

    eval_mse_r = loss_mse(data_y[:, 24:all_len, :response_size], pred_r[:, :, :response_size]).item()
    eval_mse_t = loss_mse(data_y[:, 24:all_len, -treatment_size:], pred_t[:, :, -treatment_size:]).item()
    eval_mae_r = loss_mae(data_y[:, 24:all_len, :response_size], pred_r[:, :, :response_size]).item()
    eval_mae_t = loss_mae(data_y[:, 24:all_len, -treatment_size:], pred_t[:, :, -treatment_size:]).item()
    bias=(data_y[:, 24:all_len, :response_size]-pred_r[:,:, :response_size])
    return eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t,bias,pred_r,pred_t

In [49]:
model = LSTM2(128, 1, 0).cuda()

In [11]:
model_r=LSTM(128,1,0.2,47)
model_t=LSTM(128,1,0.2,2)

c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [50]:
def simulation_rt(test_iterator, vaild_iterator, model, is_mask, window, logger, model_save_path, response_size=47, treatment_size=2,
               device=torch.device("cpu")):
    # model.load_state_dict(torch.load("trained_models/transformer/2022_08_09_13_30_10/model.pt" ,map_location={'cuda:0':'cuda:0','cuda:1': 'cuda:0','cuda:2':'cuda:0','cuda:3':'cuda:0','cuda:4':'cuda:0',
    #                                                                                                            'cuda:5':'cuda:0','cuda:6':'cuda:0','cuda:7':'cuda:0','cuda:8':'cuda:0','cuda:9':'cuda:0'}),)
    model_r=LSTM(128,1,0.2,47)
    model_t=LSTM(128,1,0.2,2)
    model_r=torch.load("model_test/LSTM_r_t_[128, 256, 512]_[1, 2]_[0.2, 0.5]_[64, 128, 256]r.model")
    model_t=torch.load("model_test/LSTM_r_t_[128, 256, 512]_[1, 2]_[0.2, 0.5]_[64, 128, 256]t.model")
    
    model_r = model_r.to(device)
    model_t = model_t.to(device)

    num_batch = 0
    valid_mse_r = 0
    valid_mse_t = 0
    valid_mae_r = 0
    valid_mae_t = 0

    noise = torch.empty(1, 24, 47).to(device)

    model_r = model_r.eval()
    model_t = model_t.eval()
    with torch.no_grad():

        for b, batch in enumerate(tqdm(vaild_iterator)):
            num_batch += 1
            data_x, mask = batch

            data_x_r = data_x[:, :24, :response_size].to(device)
            data_x_t = data_x[:, :24, -treatment_size:].to(device)
            data_x = data_x.to(device)
            mask = mask.to(device)
            data_y = data_x[:, :24, :].to(device)

            # eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t = evaluate_stacked_lstm(model, data_x_r, data_x_t, data_y, mask,
            #                                                             is_mask,
            #                                                             response_size, treatment_size)

            eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t, bias, pred_r, pred_t = evaluate_simulation_rt(model_r,model_t, data_y, data_x, mask,
                                                                                                     response_size, treatment_size)
            valid_mse_r += eval_mse_r
            valid_mse_t += eval_mse_t
            valid_mae_r += eval_mae_r
            valid_mae_t += eval_mae_t
            noise = torch.cat([noise, bias], dim=0)
        valid_mse_r /= num_batch
        valid_mse_t /= num_batch
        valid_mae_r /= num_batch
        valid_mae_t /= num_batch
        geometric_mean = np.exp(
            np.log([valid_mse_r, valid_mse_t, valid_mae_r, valid_mae_t]).mean())
        log_info = "geometric_mean {} valid_mse_response {} valid_mse_treatment {} valid_mae_response {} valid_mae_treatment {}". \
            format(round(geometric_mean, 5), round(valid_mse_r, 5), round(valid_mse_t, 5),
                   round(valid_mae_r, 5), round(valid_mae_t, 5))

    noise = noise[1:, :, :]
    print("mean:", str(noise.mean()), "std:", str(noise.std()))
    num_batch = 0
    valid_mse_r = 0
    valid_mse_t = 0
    valid_mae_r = 0
    valid_mae_t = 0
    pr_s = torch.empty(1, 24, 47).to(device)
    pr = torch.empty(1, 48, 47).to(device)
    pt = torch.empty(1, 48, 2).to(device)

    model = model.eval()
    sample_N = 100
    channal = 11
    test_mse_r = []
    test_mse_t = []
    test_mae_r = []
    test_mae_t = []
    TN=0
    TP=0
    FN=0
    FP=0
    pred_t_n=[]
    t_n=[]
    # pred_rsum=[]
    # pred_tsum=[]

    with torch.no_grad():

        for b, batch in enumerate(tqdm(test_iterator)):
            pred_rsum = torch.zeros(1, 24, 47).to(device)
            pred_tsum = torch.zeros(1, 24, 2).to(device)
            for a in range(sample_N):
                index = torch.LongTensor(random.sample(
                    range(noise.size(0)), batch[0].size(0)))
                # try:
                bias = torch.index_select(noise, 0, index.cuda())
                # bias= torch.normal(mean=0,std=0.6890,size=(bias.shape))
                # except:
                #     print("sample error")
                bias = bias.to(device)
                num_batch += 1
                data_x, mask = batch

                data_x_r = data_x[:, :24, :response_size].to(device)
                # data_x_r = data_x_r+bias
                data_x_t = data_x[:, :24, -treatment_size:].to(device)
                data_y = data_x[:,:24,:].to(device)
                mask = mask.to(device)
                data_x = data_x.to(device)

                # eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t = evaluate_stacked_lstm(model, data_x_r, data_x_t, data_y, mask,
                #                                                             is_mask,
                #                                                             response_size, treatment_size)

                
                
                #噪音实验
                eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t, bias, pred_r, pred_t = evaluate_simulation_rt(model_r,model_t, data_y, data_x, mask,
                                                                                                     response_size, treatment_size,bias=bias)
                # for i in range(batch[0].size(0)):
                #     treatment=data_x[i, 24:48, -treatment_size:].nonzero()
                #     treatment_pred=pred_t[i,:,:].nonzero()
                #     t_n.append( 0 if treatment.shape[0]==0 else 1)
                #     pred_t_n.append(0 if treatment_pred.shape[0]==0 else 1)
                #     if treatment.shape[0]==0 and treatment_pred.shape[0]==0:
                #         TN=TN+1
                #     if treatment.shape[0]!=0 and treatment_pred.shape[0]!=0:
                #         TP=TP+1  
                #     if treatment.shape[0]==0 and treatment_pred.shape[0]!=0:
                #         FP=FP+1   
                #     if treatment.shape[0]!=0 and treatment_pred.shape[0]==0:
                #         FN=FN+1                                                                
                
                # time step的实验
                # eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t, pred_r, pred_t = evaluate_timesetps(model, data_x_r, data_x_t, data_x, mask,
                #                                                                                       is_mask,
                #                                                                                       response_size, treatment_size)

                pred_rsum = pred_rsum+pred_r[:,:,:]
                pred_tsum = pred_tsum+pred_t[:,:,:]

                valid_mse_r += eval_mse_r
                valid_mse_t += eval_mse_t
                valid_mae_r += eval_mae_r
                valid_mae_t += eval_mae_t
                # if b == 6:
                #     pr_s = torch.cat([pr_s, pred_r[11:12, 24:, :]], dim=0)
                    
                #     orig = data_x[11, 0:48, channal]
                # pr=torch.cat([pr,pred_r],dim=0)
                # pt=torch.cat([pt,pred_t],dim=0)
            pred_rsum /= sample_N

            pred_tsum /= sample_N

            loss_mse = nn.MSELoss()
            loss_mae = nn.L1Loss()
            all_len = 48
            eval_mse_r = loss_mse(
                data_x[:, 24:all_len, :response_size], pred_rsum[:, :, :]).item()
            eval_mse_t = loss_mse(
                data_x[:, 24:all_len, -treatment_size:], pred_tsum[:, :, :]).item()
            eval_mae_r = loss_mae(
                data_x[:, 24:all_len, :response_size], pred_rsum[:, :, :]).item()
            eval_mae_t = loss_mae(
                data_x[:, 24:all_len, -treatment_size:], pred_tsum[:, :, :]).item()
            test_mse_r.append(eval_mse_r)
            test_mse_t.append(eval_mse_t)
            test_mae_r.append(eval_mae_r)
            test_mae_t.append(eval_mae_t)
            # fig=plt.figure()
            # matplotlib.use('TkAgg')
            # ax3 = plt.axes(projection='3d')
            # x=np.arange(0,pr.shape[1])
            # y=np.arange(0,pr.shape[2])
            # Xx, Yy = np.meshgrid(x, y)
            # Zz=data_x[1,Xx,Yy].cpu().numpy()
            # plt.xlabel("Time")
            # plt.xlabel("Channel")
            # ax3.plot_surface(Xx,Yy,Zz,rstride = 1, cstride = 1,cmap='rainbow')
            # plt.show()



        valid_mse_r /= num_batch
        valid_mse_t /= num_batch
        valid_mae_r /= num_batch
        valid_mae_t /= num_batch
        geometric_mean = np.exp(
            np.log([valid_mse_r, valid_mse_t, valid_mae_r, valid_mae_t]).mean())
        log_info = "test geometric_mean {} valid_mse_response {} valid_mse_treatment {} valid_mae_response {} valid_mae_treatment {}". \
            format(round(geometric_mean, 5), round(valid_mse_r, 5), round(valid_mse_t, 5),
                   round(valid_mae_r, 5), round(valid_mae_t, 5))
    print("MEAN LOSS valid_mse_response {} valid_mse_treatment {} valid_mae_response {} valid_mae_treatment {}".
          format(round(np.mean(test_mse_r), 5), round(np.mean(test_mse_t), 5), round(np.mean(test_mae_r), 5), round(np.mean(test_mae_t), 5)))
    print("geo_mean=", np.exp(np.log([np.mean(test_mse_r), np.mean(
        test_mse_t), np.mean(test_mae_r), np.mean(test_mae_t)]).mean()))
    print("N=", str(sample_N))

In [51]:
def simulation_RNN(test_iterator, vaild_iterator, model, is_mask, window, logger, model_save_path, response_size=47, treatment_size=2,
               device=torch.device("cpu")):

    model=torch.load("model_test\RNN123.model")
    model=torch.load("model_test\LSTM_r_t_[128, 256, 512]_[1, 2]_[0.2, 0.5]_[64, 128, 256]all.model")
    # model.load_state_dict(torch.load("trained_models/transformer/2022_07_29_03_34_19/model.pt",map_location='cuda:0'))
    model = model.to(device)

    num_batch = 0
    valid_mse_r = 0
    valid_mse_t = 0
    valid_mae_r = 0
    valid_mae_t = 0

    start = 0
    leng = 24
    end= 48

    noise = torch.empty(1, leng, 47).to(device)

    model = model.eval()
    with torch.no_grad():

        for b, batch in enumerate(tqdm(vaild_iterator)):
            num_batch += 1
            data_x, mask = batch

            data_x_r = data_x[:, :leng, :response_size].to(device)
            data_x_t = data_x[:, :leng, -treatment_size:].to(device)
            data_x = data_x.to(device)
            mask = mask.to(device)
            data_y = data_x[:, :leng, :].to(device)
            # eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t = evaluate_stacked_lstm(model, data_x_r, data_x_t, data_y, mask,
            #                                                             is_mask,
            #                                                             response_size, treatment_size)

            eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t, bias, pred = evaluate_simulation_rnn(model, data_y ,data_x, mask,
                                                                                                     
                                                                                                     response_size, treatment_size)
            valid_mse_r += eval_mse_r
            valid_mse_t += eval_mse_t
            valid_mae_r += eval_mae_r
            valid_mae_t += eval_mae_t
            noise = torch.cat([noise, bias], dim=0)
        valid_mse_r /= num_batch
        valid_mse_t /= num_batch
        valid_mae_r /= num_batch
        valid_mae_t /= num_batch
        geometric_mean = np.exp(
            np.log([valid_mse_r, valid_mse_t, valid_mae_r, valid_mae_t]).mean())
        # log_info = "geometric_mean {} valid_mse_response {} valid_mse_treatment {} valid_mae_response {} valid_mae_treatment {}". \
        #     format(round(geometric_mean, 5), round(valid_mse_r, 5), round(valid_mse_t, 5),
        #            round(valid_mae_r, 5), round(valid_mae_t, 5))

    noise = noise[1:, :, :]
    print("mean:", str(noise.mean()), "std:", str(noise.var()))
    num_batch = 0
    valid_mse_r = 0
    valid_mse_t = 0
    valid_mae_r = 0
    valid_mae_t = 0
    pr_s = torch.empty(1, leng, 47).to(device)
    pr = torch.empty(1, end, 47).to(device)
    pt = torch.empty(1, end, 2).to(device)

    model = model.eval()
    sample_N = 100
    channal = 11
    test_mse_r = []
    test_mse_t = []
    test_mae_r = []
    test_mae_t = []
    TN=0
    TP=0
    FN=0
    FP=0
    pred_t_n=[]
    t_n=[]
    # pred_rsum=[]
    # pred_tsum=[]

    with torch.no_grad():

        for b, batch in enumerate(tqdm(test_iterator)):
            pred_rsum = torch.zeros(1, leng, 47).to(device)
            pred_tsum = torch.zeros(1, leng, 2).to(device)
            for a in range(sample_N):
                index = torch.LongTensor(random.sample(
                    range(noise.size(0)), batch[0].size(0)))
                # try:
                bias = torch.index_select(noise, 0, index.cuda())
                # bias= torch.normal(mean=0,std=0.668,size=(bias.shape))  #our bias
                # bias= torch.normal(mean=0,std=0.4528,size=(bias.shape))  #Tranformer bias
                # bias= torch.normal(mean=0,std=1,size=(bias.shape))  #Informer bias
                # bias= torch.normal(mean=0,std=0.8020,size=(bias.shape))  #GRU
                # bias= torch.normal(mean=0,std=0.7803,size=(bias.shape))  #lstmRT bias
                # except:
                #     print("sample error")
                bias = bias.to(device)
                num_batch += 1
                data_x, mask = batch
                data_x=data_x.to(device)
                data_x_r = data_x[:, :leng, :response_size].to(device)
                # data_x_r = data_x_r+bias
                # data_x[:, :24, :response_size]=data_x[:, :24, :response_size]+bias
                data_x_in=data_x[:, :leng, :]
                data_x_t = data_x[:, :leng, -treatment_size:].to(device)
                # data_y = data_y.to(device)
                mask = mask.to(device)
                # data_x = data_x.to(device)

                # eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t = evaluate_stacked_lstm(model, data_x_r, data_x_t, data_y, mask,
                #                                                             is_mask,
                #                                                             response_size, treatment_size)

                
                
                #噪音实验
                eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t, _, pred = evaluate_simulation_rnn(model, data_x_in, data_x, mask,                                                                                     
                                                                                                response_size, treatment_size,bias=bias)                # for i in range(batch[0].size(0)):
                #     treatment=data_x[i, 24:48, -treatment_size:].nonzero()
                #     treatment_pred=pred_t[i,:,:].nonzero()
                #     t_n.append( 0 if treatment.shape[0]==0 else 1)
                #     pred_t_n.append(0 if treatment_pred.shape[0]==0 else 1)
                #     if treatment.shape[0]==0 and treatment_pred.shape[0]==0:
                #         TN=TN+1
                #     if treatment.shape[0]!=0 and treatment_pred.shape[0]!=0:
                #         TP=TP+1  
                #     if treatment.shape[0]==0 and treatment_pred.shape[0]!=0:
                #         FP=FP+1   
                #     if treatment.shape[0]!=0 and treatment_pred.shape[0]==0:
                #         FN=FN+1                                                                
                
                # time step的实验
                # eval_mse_r, eval_mse_t, eval_mae_r, eval_mae_t, pred_r, pred_t = evaluate_timesetps(model, data_x_r, data_x_t, data_x, mask,
                #                                                                                       is_mask,
                #                                                                                       response_size, treatment_size)

                pred_rsum = pred_rsum+pred[:, :, :response_size]
                pred_tsum = pred_tsum+pred[:, :, -treatment_size:]

                valid_mse_r += eval_mse_r
                valid_mse_t += eval_mse_t
                valid_mae_r += eval_mae_r
                valid_mae_t += eval_mae_t
                if b == 6:
                    pr_s = torch.cat([pr_s, pred[11:12, :leng, :47]], dim=0)
                    
                    orig = data_x[11, leng:end, channal]
                # pr=torch.cat([pr,pred_r],dim=0)
                # pt=torch.cat([pt,pred_t],dim=0)
            pred_rsum /= sample_N

            pred_tsum /= sample_N

            loss_mse = nn.MSELoss()
            loss_mae = nn.L1Loss()
            all_len = 48
            eval_mse_r = loss_mse(
                data_x[:, leng:all_len, :response_size], pred_rsum[:, :, :]).item()
            eval_mse_t = loss_mse(
                data_x[:, leng:all_len, -treatment_size:], pred_tsum[:, :, :]).item()
            eval_mae_r = loss_mae(
                data_x[:, leng:all_len, :response_size], pred_rsum[:, :, :]).item()
            eval_mae_t = loss_mae(
                data_x[:, leng:all_len, -treatment_size:], pred_tsum[:, :, :]).item()
            test_mse_r.append(eval_mse_r)
            test_mse_t.append(eval_mse_t)
            test_mae_r.append(eval_mae_r)
            test_mae_t.append(eval_mae_t)
            # fig=plt.figure()
            # matplotlib.use('TkAgg')
            # ax3 = plt.axes(projection='3d')
            # x=np.arange(0,pr.shape[1])
            # y=np.arange(0,pr.shape[2])
            # Xx, Yy = np.meshgrid(x, y)
            # Zz=data_x[1,Xx,Yy].cpu().numpy()
            # plt.xlabel("Time")
            # plt.xlabel("Channel")
            # ax3.plot_surface(Xx,Yy,Zz,rstride = 1, cstride = 1,cmap='rainbow')
            # plt.show()



        valid_mse_r /= num_batch
        valid_mse_t /= num_batch
        valid_mae_r /= num_batch
        valid_mae_t /= num_batch
        geometric_mean = np.exp(
            np.log([valid_mse_r, valid_mse_t, valid_mae_r, valid_mae_t]).mean())
        log_info = "test geometric_mean {} valid_mse_response {} valid_mse_treatment {} valid_mae_response {} valid_mae_treatment {}". \
            format(round(geometric_mean, 5), round(valid_mse_r, 5), round(valid_mse_t, 5),
                   round(valid_mae_r, 5), round(valid_mae_t, 5))
    print("MEAN LOSS valid_mse_response {} valid_mse_treatment {} valid_mae_response {} valid_mae_treatment {}".
          format(round(np.mean(test_mse_r), 5), round(np.mean(test_mse_t), 5), round(np.mean(test_mae_r), 5), round(np.mean(test_mae_t), 5)))
    print("geo_mean=", np.exp(np.log([np.mean(test_mse_r), np.mean(
        test_mse_t), np.mean(test_mae_r), np.mean(test_mae_t)]).mean()))
    print("N=", str(sample_N))
    # logger.info(log_info)
    # logger.info("\n")
    # logger.info("best_geometric_mean {}\n".format(round(geometric_mean, 5)))

In [52]:
model

LSTM2(
  (rnn1): LSTM(47, 128, batch_first=True)
  (rnn2): LSTM(47, 128, batch_first=True)
  (drop_layer): Dropout(p=0, inplace=False)
  (drop_layer2): Dropout(p=0, inplace=False)
  (out): Linear(in_features=128, out_features=47, bias=True)
  (out2): Linear(in_features=128, out_features=2, bias=True)
)

In [53]:
simulation_RNN( test_iterator,vaild_iterator, model, True, window=12, logger=None,
                         model_save_path=os.path.join("model_test", "rnn-model.pt"), response_size=47, treatment_size=2, device="cuda:0")

100%|██████████| 21/21 [00:01<00:00, 20.32it/s]


mean: tensor(-0.0060, device='cuda:0') std: tensor(0.5202, device='cuda:0')


100%|██████████| 21/21 [00:54<00:00,  2.62s/it]

MEAN LOSS valid_mse_response 0.48337 valid_mse_treatment 0.03196 valid_mae_response 0.4304 valid_mae_treatment 0.0687
geo_mean= 0.14619657869839497
N= 100


In [ ]:
simulation_RNN( test_iterator,vaild_iterator, model, True, window=12, logger=None,
                         model_save_path=os.path.join("model_test", "rnn-model.pt"), response_size=47, treatment_size=2, device="cuda:0")

100%|██████████| 21/21 [00:00<00:00, 51.04it/s]


mean: tensor(0.0354, device='cuda:0') std: tensor(0.7622, device='cuda:0')


100%|██████████| 21/21 [00:18<00:00,  1.11it/s]

MEAN LOSS valid_mse_response 0.6699 valid_mse_treatment 0.03885 valid_mae_response 0.46961 valid_mae_treatment 0.10547
geo_mean= 0.18948053315542857
N= 100
